In [ ]:
import numpy as np
from pykrige.ok3d import OrdinaryKriging3D
from scipy.interpolate import griddata, RBFInterpolator
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error, mean_absolute_error

Tx = np.array([5, 5, 2, 0, 1, 2, 5, 1, 2]).reshape(3, 3)
Ty = np.array([60, 70, 80, 65, 75, 65, 70, 70, 80]).reshape(3, 3)
Tz = np.array([100, 110, 105, 115, 120, 118, 125, 130, 128]).reshape(3, 3)  # 示例温度值

# 定义长方体的尺寸
Lx, Ly, Lz = 30, 50, 40

# 定义每个面的测量点和温度值
def generate_face_coords_temps(Lx, Ly, Lz, face):
    coords = []
    temps = []
    for i in range(3):
        for j in range(3):
            if face == 'bottom':
                coords.append(((2 * (i + 1) - 1) * Lx / 6, (2 * (j + 1) - 1) * Ly / 6, 0))
                temps.append(Tz[i, j])  # 示例温度值
            elif face == 'top':
                coords.append(((2 * (i + 1) - 1) * Lx / 6, (2 * (j + 1) - 1) * Ly / 6, Lz))
                temps.append(Tz[i, j])  # 示例温度值
            elif face == 'front':  # front face first row from top has same z values, so i is z
                coords.append(((2 * (j + 1) - 1) * Lx / 6, 0, (2 * (i + 1) - 1) * Lz / 6))
                temps.append(Ty[i, j])  # 示例温度值
            elif face == 'back':
                coords.append(((2 * (j + 1) - 1) * Lx / 6, Ly, (2 * (i + 1) - 1) * Lz / 6))
                temps.append(Ty[i, j])  # 示例温度值
            elif face == 'left':
                coords.append((0, (2 * (j + 1) - 1) * Ly / 6, (2 * (i + 1) - 1) * Lz / 6))
                temps.append(Tx[i, j])  # 示例温度值
            elif face == 'right':
                coords.append((Lx, (2 * (j + 1) - 1) * Ly / 6, (2 * (i + 1) - 1) * Lz / 6))
                temps.append(Tx[i, j])  # 示例温度值
    return np.array(coords), np.array(temps)

faces = ['bottom', 'top', 'front', 'back', 'left', 'right']
all_coords = []
all_temps = []

# 生成所有面的测量点
for face in faces:
    coords, temps = generate_face_coords_temps(Lx, Ly, Lz, face)
    all_coords.append(coords)
    all_temps.append(temps)

# 合并所有面的测量点
all_coords = np.vstack(all_coords)
all_temps = np.hstack(all_temps)

# 创建网格点
grid_x = np.linspace(0, Lx, 30)
grid_y = np.linspace(0, Ly, 40)
grid_z = np.linspace(0, Lz, 50)
grid_xx, grid_yy, grid_zz = np.meshgrid(grid_x, grid_y, grid_z)

# 使用三维普通克里金插值
kriging = OrdinaryKriging3D(all_coords[:, 0], all_coords[:, 1], all_coords[:, 2], all_temps, variogram_model='spherical')
grid_temps_kriging, ss = kriging.execute('grid', grid_x, grid_y, grid_z)

# 使用三维线性插值
grid_temps_linear = griddata(all_coords, all_temps, (grid_xx, grid_yy, grid_zz), method='linear')

# 使用三维最近邻插值
grid_temps_nearest = griddata(all_coords, all_temps, (grid_xx, grid_yy, grid_zz), method='nearest')

# 使用径向基函数插值
rbf_interp = RBFInterpolator(all_coords, all_temps)
grid_temps_rbf = rbf_interp(np.column_stack([grid_xx.ravel(), grid_yy.ravel(), grid_zz.ravel()])).reshape(grid_xx.shape)

# 设置颜色条范围
color_min = min(grid_temps_kriging.min(), grid_temps_linear.min(), grid_temps_nearest.min(), grid_temps_rbf.min())
color_max = max(grid_temps_kriging.max(), grid_temps_linear.max(), grid_temps_nearest.max(), grid_temps_rbf.max())

# 坐标轴范围设置
x_range = [0, Lx]
y_range = [0, Ly]
z_range = [0, Lz]

# 可视化结果
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}]],
    subplot_titles=(f'Original data',
                    f'3D Kriging Interpolation ',
                    f'3D Linear Interpolation ',
                    # f'3D Nearest Interpolation ',
                    f'3D RBF Interpolation ')
)

# # 原始测量值图
trace1 = go.Scatter3d(
    x=all_coords[:, 0],
    y=all_coords[:, 1],
    z=all_coords[:, 2],
    mode='markers',
    marker=dict(
        size=10,
        color=all_temps,
        colorscale='plasma_r',
        symbol='circle',
        cmin=color_min,
        cmax=color_max
    ),
    # text=[f'{temp}' for temp in all_temps],
    # textposition='top center',
    # name='Original Data'
)

# 3D 克里金插值图
trace2 = go.Volume(
    x=grid_xx.ravel(),
    y=grid_yy.ravel(),
    z=grid_zz.ravel(),
    value=grid_temps_kriging.ravel(),
    isomin=color_min,
    isomax=color_max,
    opacity=0.1,
    surface_count=10,
    colorscale='plasma_r',
    colorbar=dict(title='Temperature')
)

# 3D 线性插值图
trace3 = go.Volume(
    x=grid_xx.ravel(),
    y=grid_yy.ravel(),
    z=grid_zz.ravel(),
    value=grid_temps_linear.ravel(),
    isomin=color_min,
    isomax=color_max,
    opacity=0.1,
    surface_count=10,
    colorscale='plasma_r',
    colorbar=dict(title='Temperature')
)

# 3D 最近邻插值图
# trace3 = go.Volume(
#     x=grid_xx.ravel(),
#     y=grid_yy.ravel(),
#     z=grid_zz.ravel(),
#     value=grid_temps_nearest.ravel(),
#     isomin=color_min,
#     isomax=color_max,
#     opacity=0.1,
#     surface_count=10,
#     colorscale='plasma_r',
#     colorbar=dict(title='Temperature')
# )

# 3D RBF 插值图
trace4 = go.Volume(
    x=grid_xx.ravel(),
    y=grid_yy.ravel(),
    z=grid_zz.ravel(),
    value=grid_temps_rbf.ravel(),
    isomin=color_min,
    isomax=color_max,
    opacity=0.1,
    surface_count=10,
    colorscale='plasma_r',
    colorbar=dict(title='Temperature')
)

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
fig.add_trace(trace3, row=2, col=1)
fig.add_trace(trace4, row=2, col=2)

# 更新布局
fig.update_layout(
    height=900, width=1200,
    title_text="Comparison of 3D Kriging and Various Interpolation Methods",
    scene=dict(
        xaxis=dict(range=x_range),
        yaxis=dict(range=y_range),
        zaxis=dict(range=z_range),
        aspectmode='cube'
    ),
    scene2=dict(
        xaxis=dict(range=x_range),
        yaxis=dict(range=y_range),
        zaxis=dict(range=z_range),
        aspectmode='cube'
    ),
    scene3=dict(
        xaxis=dict(range=x_range),
        yaxis=dict(range=y_range),
        zaxis=dict(range=z_range),
        aspectmode='cube'
    ),
    scene4=dict(
        xaxis=dict(range=x_range),
        yaxis=dict(range=y_range),
        zaxis=dict(range=z_range),
        aspectmode='cube'
    )
)
fig.show()

# 将图表保存为HTML文件
html_content = fig.to_html(full_html=False)

# 添加JavaScript代码同步旋转
html_content += """
<script>
    var gd = document.getElementById('plotly-div');
    
    var camera;
    
    function syncCamera(eventdata) {
        var scene = gd._fullLayout.scene._scene;
        if (eventdata['scene.camera']) {
            camera = eventdata['scene.camera'];
        } else if (camera) {
            Plotly.relayout(gd, {
                'scene.camera': camera,
                'scene2.camera': camera,
                'scene3.camera': camera,
                'scene4.camera': camera
            });
        }
    }

    gd.on('plotly_relayout', syncCamera);
</script>
"""

with open('plot_3D.html', 'w') as f:
    f.write(html_content)

print("Plot saved to plot_3D.html")


In [ ]:
import numpy as np
from pykrige.ok import OrdinaryKriging
from scipy.interpolate import griddata, RBFInterpolator
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

Tx = np.array([-5, -5, -2, 0, -1, -2, -5, -1, -2]).reshape(3, 3)
Ty = np.array([60, 70, 80, 65, 75, 65, 70, 70, 80]).reshape(3, 3)
Tz = np.array([100, 110, 105, 115, 120, 118, 125, 130, 128]).reshape(3, 3)  # 示例温度值

# 定义长方体的尺寸
Lx, Ly, Lz = 30, 50, 40

# 定义每个面的测量点和温度值
def generate_face_coords_temps(Lx, Ly, Lz, face):
    coords = []
    temps = []
    for i in range(3):
        for j in range(3):
            if face == 'bottom':
                coords.append(((2 * (i + 1) - 1) * Lx / 6, (2 * (j + 1) - 1) * Ly / 6, 0))
                temps.append(Tz[i, j])  # 示例温度值
            elif face == 'top':
                coords.append(((2 * (i + 1) - 1) * Lx / 6, (2 * (j + 1) - 1) * Ly / 6, Lz))
                temps.append(Tz[i, j])  # 示例温度值
            elif face == 'front':  # front face first row from top has same z values, so i is z
                coords.append(((2 * (j + 1) - 1) * Lx / 6, 0, (2 * (i + 1) - 1) * Lz / 6))
                temps.append(Ty[i, j])  # 示例温度值
            elif face == 'back':
                coords.append(((2 * (j + 1) - 1) * Lx / 6, Ly, (2 * (i + 1) - 1) * Lz / 6))
                temps.append(Ty[i, j])  # 示例温度值
            elif face == 'left':
                coords.append((0, (2 * (j + 1) - 1) * Ly / 6, (2 * (i + 1) - 1) * Lz / 6))
                temps.append(Tx[i, j])  # 示例温度值
            elif face == 'right':
                coords.append((Lx, (2 * (j + 1) - 1) * Ly / 6, (2 * (i + 1) - 1) * Lz / 6))
                temps.append(Tx[i, j])  # 示例温度值
    return np.array(coords), np.array(temps)

# 选择需要插值的面
selected_face = 'bottom'  # 可以改为 'top', 'front', 'back', 'left', 'right'

# 生成所选面的测量点
coords, temps = generate_face_coords_temps(Lx, Ly, Lz, selected_face)

# 添加微小噪声以避免共面性问题
coords += np.random.normal(0, 1e-5, coords.shape)

# 创建网格点
if selected_face in ['bottom', 'top']:
    grid_x = np.linspace(0, Lx, 100)
    grid_y = np.linspace(0, Ly, 100)
elif selected_face in ['front', 'back']:
    grid_x = np.linspace(0, Lx, 100)
    grid_y = np.linspace(0, Lz, 100)
else:  # 'left', 'right'
    grid_x = np.linspace(0, Ly, 100)
    grid_y = np.linspace(0, Lz, 100)
grid_xx, grid_yy = np.meshgrid(grid_x, grid_y)

# 使用二维普通克里金插值
kriging_2d = OrdinaryKriging(coords[:, 0], coords[:, 1], temps, variogram_model='spherical')
grid_temps_kriging_2d, ss_2d = kriging_2d.execute('grid', grid_x, grid_y)

# 使用二维线性插值
grid_temps_linear_2d = griddata(coords[:, :2], temps, (grid_xx, grid_yy), method='linear')

# 使用二维最近邻插值
grid_temps_nearest_2d = griddata(coords[:, :2], temps, (grid_xx, grid_yy), method='nearest')

# 使用二维RBF插值
rbf_2d = RBFInterpolator(coords[:, :2], temps)
grid_temps_rbf_2d = rbf_2d(np.column_stack([grid_xx.ravel(), grid_yy.ravel()])).reshape(grid_xx.shape)

# 设置颜色条范围
color_min = min(grid_temps_kriging_2d.min(), grid_temps_linear_2d.min(), grid_temps_nearest_2d.min(), grid_temps_rbf_2d.min())
color_max = max(grid_temps_kriging_2d.max(), grid_temps_linear_2d.max(), grid_temps_nearest_2d.max(), grid_temps_rbf_2d.max())

# 使用Plotly可视化结果
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}]],
    subplot_titles=('2D Kriging Interpolation',
                    '2D Linear Interpolation',
                    '2D Cubic Interpolation',
                    '2D RBF Interpolation')
)

# 2D 克里金插值图
trace1 = go.Surface(
    x=grid_x,
    y=grid_y,
    z=grid_temps_kriging_2d,
    colorscale='Viridis',
    cmin=color_min,
    cmax=color_max,
    colorbar=dict(title='Temperature')
)

# 2D 线性插值图
trace2 = go.Surface(
    x=grid_x,
    y=grid_y,
    z=grid_temps_linear_2d,
    colorscale='Viridis',
    cmin=color_min,
    cmax=color_max,
    colorbar=dict(title='Temperature')
)

# 2D 最近邻插值图
trace3 = go.Surface(
    x=grid_x,
    y=grid_y,
    z=grid_temps_nearest_2d,
    colorscale='Viridis',
    cmin=color_min,
    cmax=color_max,
    colorbar=dict(title='Temperature')
)

# 2D RBF 插值图
trace4 = go.Surface(
    x=grid_x,
    y=grid_y,
    z=grid_temps_rbf_2d,
    colorscale='Viridis',
    cmin=color_min,
    cmax=color_max,
    colorbar=dict(title='Temperature')
)

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
fig.add_trace(trace3, row=2, col=1)
fig.add_trace(trace4, row=2, col=2)

# 更新布局
fig.update_layout(
    height=900, width=1200,
    title_text="Comparison of 2D Kriging, Linear, Nearest, and RBF Interpolation",
    scene=dict(
        aspectmode='cube'
    ),
    scene2=dict(
        aspectmode='cube'
    ),
    scene3=dict(
        aspectmode='cube'
    ),
    scene4=dict(
        aspectmode='cube'
    )
)

# 将图表保存为HTML文件
html_content = fig.to_html(full_html=False)

# 添加JavaScript代码同步旋转
html_content += """
<script>
    var gd = document.getElementById('plotly-div');

    var camera;

    function syncCamera(eventdata) {
        if (eventdata['scene.camera']) {
            camera = eventdata['scene.camera'];
            Plotly.relayout(gd, {
                'scene.camera': camera,
                'scene2.camera': camera,
                'scene3.camera': camera,
                'scene4.camera': camera
            });
        }
    }

    gd.on('plotly_relayout', syncCamera);
</script>
"""

with open('plot_2D.html', 'w') as f:
    f.write(html_content)

print("Plot saved to plot_2D.html")

# 使用imshow生成对比图
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# 2D 克里金插值图
im1 = axs[0, 0].imshow(grid_temps_kriging_2d, extent=(0, Lx, 0, Ly), origin='lower', cmap='viridis', vmin=color_min, vmax=color_max)
axs[0, 0].set_title('2D Kriging Interpolation')
plt.colorbar(im1, ax=axs[0, 0])

# 2D 线性插值图
im2 = axs[0, 1].imshow(grid_temps_linear_2d, extent=(0, Lx, 0, Ly), origin='lower', cmap='viridis', vmin=color_min, vmax=color_max)
axs[0, 1].set_title('2D Linear Interpolation')
plt.colorbar(im2, ax=axs[0, 1])

# 2D 最近邻插值图
im3 = axs[1, 0].imshow(grid_temps_nearest_2d, extent=(0, Lx, 0, Ly), origin='lower', cmap='viridis', vmin=color_min, vmax=color_max)
axs[1, 0].set_title('2D Nearest Interpolation')
plt.colorbar(im3, ax=axs[1, 0])

# 2D RBF 插值图
im4 = axs[1, 1].imshow(grid_temps_rbf_2d, extent=(0, Lx, 0, Ly), origin='lower', cmap='viridis', vmin=color_min, vmax=color_max)
axs[1, 1].set_title('2D RBF Interpolation')
plt.colorbar(im4, ax=axs[1, 1])

plt.tight_layout()
plt.show()
